In [1]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the main directory containing leaf, fruit, flower, and bark folders
main_directory = os.getcwd()

# Exclude the "json" folder from the list of classes
classes = [class_name for class_name in os.listdir(main_directory) if os.path.isdir(os.path.join(main_directory, class_name)) and class_name != 'json']

# Set up data generators for training
train_datagen = ImageDataGenerator(validation_split=0.2)  # Remove rescale parameter

# Define the target size and batch size
target_size = (250, 250)  # All images are scaled to 250 x 250
batch_size = 32

# Load training data
train_generator = train_datagen.flow_from_directory(
    main_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',  # use 'binary' if you have only two classes
    subset='training',
    classes=classes
)

# Load validation data
validation_generator = train_datagen.flow_from_directory(
    main_directory,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    classes=classes
)


Found 780960 images belonging to 4 classes.
Found 195237 images belonging to 4 classes.


In [3]:
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Build a simple CNN model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))  # 4 classes: 'leaf', 'fruit', 'flower', 'bark'

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Define model checkpoint
model_checkpoint = ModelCheckpoint('model_checkpoint.h5', save_best_only=True)

# Train the model with early stopping and model checkpoint
epochs = 100  # Increase the number of epochs for demonstration
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[early_stopping, model_checkpoint])

Epoch 1/100
    7/24405 [..............................] - ETA: 9:04:33 - loss: 656.9799 - accuracy: 0.2500

KeyboardInterrupt: 

In [4]:
from tensorflow.keras.models import load_model

# Specify the checkpoint file
checkpoint_file = 'model_checkpoint.h5'

# Load the model from the checkpoint file
model = load_model(checkpoint_file)
print("Model loaded from checkpoint.")


# Compile the model (you need to compile it again after loading the weights)
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with early stopping and model checkpoint
epochs = 100  # Increase the number of epochs for demonstration
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[early_stopping, model_checkpoint])


Model loaded from checkpoint.
Epoch 1/100
   46/24405 [..............................] - ETA: 8:16:42 - loss: 1.2608 - accuracy: 0.3981

#############################################################################################################3

In [ ]:
# OLD without ModelCheckpoint
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping

# Build a simple CNN model
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))  # 4 classes: 'leaf', 'fruit', 'flower', 'bark'

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
epochs = 100  # Increase the number of epochs for demonstration
history = model.fit(train_generator, epochs=epochs, validation_data=validation_generator, callbacks=[early_stopping])